In [8]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup as bs4
import requests
import pandas as pd
from time import sleep
from random import randint
import time

creating connection

In [9]:
secrets_file = open("secrets.txt","r")
string = secrets_file.read()
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

preparing functions for the big war

In [10]:
def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,3))
    return tracks

In [11]:
def get_name(song):
    name = song["track"]["name"]
    return name
def get_id(song):
    return song["track"]["id"]
def get_uri(song):
    return song["track"]["uri"]
def get_duration(song):
    return song["track"]["duration_ms"]
def get_artists(song):
    all_artist = []
    artists = song["track"]["artists"]
    for i in artists:
        artist = i["name"]
        all_artist.append(artist)
    artists = ", ".join(all_artist)
    return artists

empty list to keep all tracks obtained from playlists

In [12]:
all_tracks = []

put all playlists you want to get songs from

In [13]:
playlists = ["3LgwHwHUnheGm88n8pDdA7","2JHheevZhehi18oX4t4lXE","1hMzceeWw7QiI6vaBkcEJO",
"4rnleEAOdmFAbRcNCgZMpY","69fEt9DN5r4JQATi52sRtq","5S8SJdl1BDc0ugpkEvFsIL","7ikZLGjkIILujRBn2GL5bM",
"37i9dQZF1DX4PP3DA4J0N8","0vvXsWCC9xrXsKd4FyS8kM","4rnleEAOdmFAbRcNCgZMpY","2YRe7HRKNRvXdJBp9nXFza",
"5xqpyfZyS1DVysoevdVyEn"]

start collecting songs from playlists. this might take long, depending on the total song number

In [14]:
for i in playlists:
    tracks = get_playlist_tracks(i)
    all_tracks = all_tracks + tracks
    time.sleep(10)
len(all_tracks)

48997

run functions to get informations from track

In [15]:
ltitle = []
lid = []
luri = []
lduration = []
lartists = []
for i in all_tracks:
    songtitle = get_name(i)
    songid = get_id(i)
    songuri = get_uri(i)
    songduration = get_duration(i)
    songartists = get_artists(i)
    ltitle.append(songtitle)
    lid.append(songid)
    luri.append(songuri)
    lduration.append(songduration)
    lartists.append(songartists)
data = pd.DataFrame({"title":ltitle, "id":lid, "uri":luri, "duration_s":lduration, "artists":lartists})
data.head()

,title,id,uri,duration_s,artists
0,Livin' On A Prayer,37ZJ0p5Jm13JPevGcx4SkF,spotify:track:37ZJ0p5Jm13JPevGcx4SkF,249293,Bon Jovi
1,Wishing on a Star,0u2CGItUO2jYEiFaXD7n5S,spotify:track:0u2CGItUO2jYEiFaXD7n5S,291760,Rose Royce
2,Supremacy,40pPI2TbaYSZlKfV44HRjn,spotify:track:40pPI2TbaYSZlKfV44HRjn,295373,Muse
3,Saturday Night - Radio Mix,59VRFpPnC8pOhIH2WCWXF9,spotify:track:59VRFpPnC8pOhIH2WCWXF9,247405,Whigfield
4,The Real Slim Shady,3yfqSUWxFvZELEM4PmlwIR,spotify:track:3yfqSUWxFvZELEM4PmlwIR,284200,Eminem


sample

In [16]:
all_tracks[32000]["track"]["name"]

'turn ways'

now get all audio_features from each collected track

In [17]:
uri = data.uri.tolist()
df_uri = pd.DataFrame()
for i in uri:
    songuri = sp.audio_features(i)
    songuri = pd.DataFrame(songuri)
    df_uri = pd.concat([df_uri, songuri])

Expected id of type track but found type In+the+Shadow+of+the+Valley spotify:local:::In+the+Shadow+of+the+Valley:187
Expected id of type track but found type April+Sweatpants spotify:local:::April+Sweatpants:166
Expected id of type track but found type Gee+%28Korean+Ver.+MP3+only%29 spotify:local:SNSD::Gee+%28Korean+Ver.+MP3+only%29:205
Expected id of type track but found type Hero spotify:local:::Hero:194
Expected id of type track but found type Beneath+the+Mask+%5BWith+Lyrics%5D+-+Persona+5 spotify:local:::Beneath+the+Mask+%5BWith+Lyrics%5D+-+Persona+5:285
Expected id of type track but found type Last+Surprise+%5BWith+Lyrics%5D+-+Persona+5 spotify:local:::Last+Surprise+%5BWith+Lyrics%5D+-+Persona+5:236
Expected id of type track but found type Vampire+Weekend++Ottoman spotify:local:::Vampire+Weekend++Ottoman:244
Expected id of type track but found type Vampire+Weekend+-+Ottoman spotify:local:::Vampire+Weekend+-+Ottoman:234
Expected id of type track but found type Father+Christmas spot

cleaning up a bit

In [18]:
df_uri = df_uri.reset_index()
df_uri = df_uri.drop(["type",0, "index"], axis = 1)
data = data.drop(["id","uri","duration_s"], axis = 1)

In [19]:
display(df_uri.shape)
display(data.shape)

(48997, 17)

(48997, 2)

combine titles+artists with audio_features

In [20]:
full = pd.concat([data,df_uri], axis = 1)
full.head()

,title,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Livin' On A Prayer,Bon Jovi,0.532,0.887,0.0,-3.757,1.0,0.0335,0.07780,0.000214,0.2940,0.795,122.511,37ZJ0p5Jm13JPevGcx4SkF,spotify:track:37ZJ0p5Jm13JPevGcx4SkF,https://api.spotify.com/v1/tracks/37ZJ0p5Jm13J...,https://api.spotify.com/v1/audio-analysis/37ZJ...,249293.0,4.0
1,Wishing on a Star,Rose Royce,0.335,0.422,5.0,-15.172,0.0,0.0519,0.71600,0.000040,0.0455,0.553,97.660,0u2CGItUO2jYEiFaXD7n5S,spotify:track:0u2CGItUO2jYEiFaXD7n5S,https://api.spotify.com/v1/tracks/0u2CGItUO2jY...,https://api.spotify.com/v1/audio-analysis/0u2C...,291760.0,4.0
2,Supremacy,Muse,0.256,0.679,9.0,-4.495,0.0,0.0387,0.00451,0.040900,0.1720,0.325,158.718,40pPI2TbaYSZlKfV44HRjn,spotify:track:40pPI2TbaYSZlKfV44HRjn,https://api.spotify.com/v1/tracks/40pPI2TbaYSZ...,https://api.spotify.com/v1/audio-analysis/40pP...,295373.0,4.0
3,Saturday Night - Radio Mix,Whigfield,0.728,0.784,11.0,-11.776,1.0,0.0379,0.04490,0.091800,0.0964,0.964,129.981,59VRFpPnC8pOhIH2WCWXF9,spotify:track:59VRFpPnC8pOhIH2WCWXF9,https://api.spotify.com/v1/tracks/59VRFpPnC8pO...,https://api.spotify.com/v1/audio-analysis/59VR...,247406.0,4.0
4,The Real Slim Shady,Eminem,0.949,0.661,5.0,-4.244,0.0,0.0572,0.03020,0.000000,0.0454,0.760,104.504,3yfqSUWxFvZELEM4PmlwIR,spotify:track:3yfqSUWxFvZELEM4PmlwIR,https://api.spotify.com/v1/tracks/3yfqSUWxFvZE...,https://api.spotify.com/v1/audio-analysis/3yfq...,284200.0,4.0


In [21]:
display(full.shape)
full = full.drop_duplicates(subset = "id")
full.shape

(48997, 19)

(41816, 19)

all clear

In [22]:
full.to_csv("songlists.csv", index = False)